In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime
import numpy as np
import re
#import openai
#import pymupdf

import os
from pathlib import Path
os.environ['USER_AGENT'] = 'myagent'

os.chdir("/Users/roamingt/Documents/Python Projects")
cwd = Path.cwd()
print(cwd)

from dotenv import load_dotenv, dotenv_values
load_dotenv()
openai_api_key=os.getenv('openai_api_key')
os.getenv('openai_api_key')
#print(temp)

from langchain_community.document_loaders import WebBaseLoader

from langchain_openai import ChatOpenAI
#from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain.document_loaders import TextLoader



/Users/roamingt/Documents/Python Projects


In [ ]:
"""
List of publications:
https://www.occ.treas.gov/publications-and-resources/publications/consumer-protection-pubs/index-consumer-protection.html
https://www.occ.treas.gov/publications-and-resources/publications/community-affairs/index-community-affairs-publications.html
https://www.federalreserve.gov/supervisionreg/dfa-stress-tests-2025.htm
https://www.federalreserve.gov/monetarypolicy/publications/mpr_default.htm
https://www.federalreserve.gov/monetarypolicy/publications/beige-book-default.htm
https://www.fdic.gov/news/financial-institution-letters/index.html
https://ncua.gov/regulation-supervision/letters-credit-unions-other-guidance
https://www.sec.gov/rules-regulations/rulemaking-activity
https://www.cftc.gov/learnandprotect/learningresources


https://www.consumerfinance.gov/compliance/supervisory-highlights/


In [10]:
# URL of the CFPB Supervisory Highlights page
url = "https://www.consumerfinance.gov/compliance/supervisory-highlights/"

# Fetch the page content
response = requests.get(url)
response.raise_for_status()

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

# Select all the <a> tags with the specific class for links
link_elements = soup.select('a.a-link.a-link--jump')

# Extract titles and links
data = []
for link in link_elements:
    title_element = link.select_one('span.a-link__text')
    
    if title_element:
        title = title_element.get_text(strip=True)
        href = link.get('href')
        
        # If href is relative, prepend the domain
        if href.startswith('/'):
            href = 'https://www.consumerfinance.gov' + href
        
        data.append({
            'Title': title,
            'Link': href
        })

# Convert to a DataFrame
df = pd.DataFrame(data)

# Output the DataFrame
print(df)

# Optional: Save to CSV
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
df.to_csv(f'supervisory_highlights_{timestamp}.csv', index=False)


                                                Title  \
0   Advanced Technologies Special Edition, Issue 3...   
1   Supervisory Highlights: Special Edition Studen...   
2      Supervisory Highlights, Issue 37 (Winter 2024)   
3   Supervisory Highlights: Auto Finance Special E...   
4   Supervisory Highlights: Servicing and Collecti...   
5       Supervisory Highlights, Issue 32, Spring 2024   
6       Supervisory Highlights, Issue 33, Spring 2024   
7   Supervisory Highlights Junk Fees Update Specia...   
8         Download Summer 2023 Supervisory Highlights   
9   Download Supervisory Highlights Special Editio...   
10          Download Fall 2022 Supervisory Highlights   
11  Download Supervisory Highlights Student Loan S...   
12        Download Spring 2022 Supervisory Highlights   
13          Download Fall 2021 Supervisory Highlights   
14        Download Summer 2021 Supervisory Highlights   
15  Download COVID-19 Prioritized Assessments Spec...   
16        Download Winter 2020 

In [12]:
pd.set_option('display.max_colwidth', None)
print(df)

                                                                                        Title  \
0                               Advanced Technologies Special Edition, Issue 38 (Winter 2025)   
1             Supervisory Highlights: Special Edition Student Lending, Issue 36 (Winter 2024)   
2                                              Supervisory Highlights, Issue 37 (Winter 2024)   
3                  Supervisory Highlights: Auto Finance Special Edition, Issue 35 (Fall 2024)   
4   Supervisory Highlights: Servicing and Collection of Consumer Debt, Issue 34 (Summer 2024)   
5                                               Supervisory Highlights, Issue 32, Spring 2024   
6                                               Supervisory Highlights, Issue 33, Spring 2024   
7                Supervisory Highlights Junk Fees Update Special Edition, Issue 31, Fall 2023   
8                                                 Download Summer 2023 Supervisory Highlights   
9                             

In [ ]:
https://files.consumerfinance.gov/f/documents/cfpb_supervisory-highlights-special-ed-auto-finance_2024-10.pdf
https://files.consumerfinance.gov/f/documents/cfpb_supervisory-highlights-special-ed-auto-finance_2024-10.pdf

In [ ]:
# Function to scrape FINRA Notices
def scrape_finra_notices(start_date, end_date):
    url = "https://www.finra.org/rules-guidance/notices"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print("Failed to fetch FINRA Notices page.")
        return pd.DataFrame()  # Return an empty DataFrame if request fails
    
    soup = BeautifulSoup(response.text, "html.parser")
    rows = soup.select("table tbody tr")
    
    data = []
    
    for row in rows:
        cols = row.find_all("td")
        if len(cols) < 3:  # Ensure we have enough columns
            continue  # Skip malformed rows
        
        # Extract date from first column
        date_tag = cols[0].find("time")
        if not date_tag:
            continue  # Skip rows without a date
        date_str = date_tag["datetime"].split("T")[0]  # Extract "YYYY-MM-DD"
        date_issued = datetime.datetime.strptime(date_str, "%Y-%m-%d").date()

        # Extract title & link from the second column
        link_tag = cols[1].find("a")
        title = link_tag.text.strip() if link_tag else "No Title"
        link = "https://www.finra.org" + link_tag["href"] if link_tag else None

        # Extract full notice description from third column
        description = cols[2].text.strip() if len(cols) > 2 else ""

        # Filter by date range
        if start_date <= date_issued <= end_date:
            data.append([date_issued, title, link, description])

    # Convert data to DataFrame
    df = pd.DataFrame(data, columns=["Date", "Title", "Link", "Description"])
    
    return df

In [ ]:

# Function to fetch full text from notice links
def fetch_notice_text(url):
    """Fetch and clean the main content of a FINRA notice."""
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Failed to fetch notice: {url}")
        return None
    
    soup = BeautifulSoup(response.text, "html.parser")

    # Locate the main content area (adjust selector as needed)
    content_sections = soup.select("div.region-content, div#block-system-main, div.article-body")  

    # Extract text while preserving some structure
    paragraphs = []
    for section in content_sections:
        for p in section.find_all(["p", "h2", "h3", "ul", "li"]):  # Get paragraphs, headers, lists
            text = p.get_text(separator=" ", strip=True)
            if text:
                paragraphs.append(text)

    # Clean up text (remove excess whitespace, format headers/lists)
    structured_text = "\n\n".join(paragraphs)
    
    return structured_text if structured_text else "No content extracted."